In [1]:
import pickle
import pandas as pd
import os
import numpy as np
from itertools import combinations

author_code = r'/Users/Russell/Dropbox/MFE_Courses/Deep_Learning/Final_project/author_code/ADGAT/'
wrds_data = r'/Users/Russell/Dropbox/MFE_Courses/Deep_Learning/Final_project/project/raw_Data'
save_location = r'/Users/Russell/Dropbox/MFE_Courses/Deep_Learning/Final_project/project/Data'

In [2]:
mapping = pd.read_csv(os.path.join(author_code, 'raw_data/relation/sp500_industry.tsv'), sep = '\t')

In [3]:
mapping['ticker'] = mapping['Exchange:Ticker'].apply( lambda x: x.split(':')[1])

map_dict = mapping.set_index('CIQID').to_dict()['ticker']
mapping_list = mapping['ticker'].to_list()

### Generate x_numerical and y_

In [4]:
#### load pnl and extract the corresponding ticker

price = pd.read_csv(os.path.join(wrds_data, 'crsp_stock_price.csv'))

price['turnover'] = price['VOL']/(price['SHROUT']*1000)
price['date'] = pd.to_datetime(price['date'], format = '%Y%m%d')

spx = pd.read_csv(os.path.join(wrds_data, 'spx.csv'))

/Users/Russell/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
price.head()

,PERMNO,date,TICKER,COMNAM,PERMCO,CUSIP,SHRFLG,BIDLO,ASKHI,PRC,VOL,BID,ASK,SHROUT,OPENPRC,NUMTRD,turnover
0,10104,2015-01-02,ORCL,ORACLE CORP,8045,68389X10,NaN,43.97,45.1898,44.33,15070165.0,44.33,44.34,4391367.0,45.02,NaN,0.003432
1,10104,2015-01-05,ORCL,ORACLE CORP,8045,68389X10,0.0,43.58,44.2500,43.59,18375801.0,43.60,43.61,4391367.0,44.16,NaN,0.004185
2,10104,2015-01-06,ORCL,ORACLE CORP,8045,68389X10,NaN,42.99,44.1800,43.14,19229461.0,43.11,43.12,4391367.0,44.06,NaN,0.004379
3,10104,2015-01-07,ORCL,ORACLE CORP,8045,68389X10,NaN,43.01,43.5200,43.15,13502164.0,43.12,43.13,4391367.0,43.33,NaN,0.003075
4,10104,2015-01-08,ORCL,ORACLE CORP,8045,68389X10,NaN,43.38,43.9400,43.41,17516877.0,43.43,43.44,4391367.0,43.63,NaN,0.003989


In [6]:

res_list = []
ticker_list = []
y_list = []

valid_count = 0
for ticker in mapping_list:
    
    df = price[price['TICKER']==ticker].copy()
    
    df['date'].sort_values(ascending=True)
    df['p_low'] = df['BIDLO'].diff()/df['BIDLO'].shift()
    df['p_high'] = df['ASKHI'].diff()/df['ASKHI'].shift()
    df['p_open'] = df['OPENPRC'].diff()/df['OPENPRC'].shift()
    df['p_close'] = df['PRC'].diff()/df['PRC'].shift()
    df['p_close_next_day'] = df['p_close'].shift(-1)
    temp = df[['p_low','p_high','p_open','p_close','turnover','p_close_next_day']].dropna()
    if temp.shape[0] == spx.shape[0]-2:
        print(f"{ticker} Acceptable")
        res_list.append(temp[['p_low','p_high','p_open','p_close','turnover']])
        y_list.append(temp['p_close_next_day'])
        ticker_list.append(ticker)
        valid_count +=1

print(valid_count)

MMM Acceptable
AOS Acceptable
ABT Acceptable
ABBV Acceptable
ABMD Acceptable
ACN Acceptable
ATVI Acceptable
ADBE Acceptable
AAP Acceptable
AMD Acceptable
AFL Acceptable
A Acceptable
APD Acceptable
AKAM Acceptable
ALK Acceptable
ALB Acceptable
ARE Acceptable
ALGN Acceptable
ALLE Acceptable
LNT Acceptable
GOOG Acceptable
MO Acceptable
AMZN Acceptable
AEE Acceptable
AAL Acceptable
AEP Acceptable
AXP Acceptable
AIG Acceptable
AMT Acceptable
AWK Acceptable
AMP Acceptable
ABC Acceptable
AME Acceptable
AMGN Acceptable
APH Acceptable
ADI Acceptable
ANSS Acceptable
ANTM Acceptable
AON Acceptable
APA Acceptable
AAPL Acceptable
AMAT Acceptable
ADM Acceptable
ANET Acceptable
AJG Acceptable
AIZ Acceptable
T Acceptable
ATO Acceptable
ADSK Acceptable
ADP Acceptable
AZO Acceptable
AVB Acceptable
AVY Acceptable
BLL Acceptable
BAC Acceptable
BAX Acceptable
BDX Acceptable
BBY Acceptable
BLK Acceptable
BWA Acceptable
BXP Acceptable
BSX Acceptable
BMY Acceptable
AVGO Acceptable
BR Acceptable
CHRW Acceptabl

In [7]:
x_numerical = np.array(res_list) #### (firm, dates, prices)

x_numerical = x_numerical.transpose((1,0,2))

x_numerical.shape

(1509, 424, 5)

In [8]:
fileObject = open(os.path.join(save_location,
                              'x_numerical.pkl'), 'wb')

pickle.dump(x_numerical,fileObject)
fileObject.close()

In [9]:
y_ = np.array(y_list)

print(y_.shape)

y_ = y_.transpose((1,0))

fileObject = open(os.path.join(save_location, 'y_.pkl'), 'wb')

pickle.dump(y_,fileObject)
fileObject.close()

(424, 1509)


# Find the corresponding 

### Industry

In [10]:
res_ind = pickle.load(open(os.path.join(author_code, 'data/relations/industry_relation.pkl'),'rb'))

In [11]:
res_ind

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
ind_map = pd.read_csv(os.path.join(author_code, 'raw_data/relation/sp500_industry.tsv'), sep = '\t')

In [13]:
ind_map['ticker'] = ind_map['Exchange:Ticker'].apply( lambda x: x.split(':')[1])
ind_map = ind_map[ind_map['ticker'].isin(ticker_list)].copy()
ind_map = ind_map.set_index('ticker')
ind_map = ind_map.loc[ticker_list]
ind_map['idx']  = 1
ind_map['idx'] = ind_map['idx'].cumsum()-1

In [14]:
ind_map.shape

(424, 5)

In [15]:
ind_res = np.zeros((valid_count, valid_count))

for cat, df in ind_map.groupby('Primary Industry'):
    #### fill in industry with same industry as 1
    for x,y in combinations(df['idx'].unique(),2):
        ind_res[x,y] = 1
        ind_res[y,x] = 1
        
fileObject = open(os.path.join(save_location, 'relations_author_source/industry_relation.pkl'), 'wb')

pickle.dump(ind_res,fileObject)
fileObject.close()

In [16]:
ind_res.shape

(424, 424)

In [17]:
ind_res

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Supply Train

In [18]:
res_supply = pickle.load(open(os.path.join(author_code, 'data/relations/supply_relation.pkl'),'rb'))

In [19]:
supply = pd.read_csv(os.path.join(author_code, 'raw_data/relation/sp500_supply.tsv'), sep = '\t')

In [20]:
supply.head()

,NYSE:MMM,NYSE:AOS,NYSE:ABT,NYSE:ABBV,NasdaqGS:ABMD,NYSE:ACN,NasdaqGS:ATVI,NasdaqGS:ADBE,NYSE:AAP,NasdaqGS:AMD,...,NasdaqGS:WYNN,NasdaqGS:XEL,NYSE:XRX,NasdaqGS:XLNX,NYSE:XYL,NYSE:YUM,NasdaqGS:ZBRA,NYSE:ZBH,NasdaqGS:ZION,NYSE:ZTS
0,IQ881351,IQ250510,IQ303225632,IQ3182703,IQ28046477,IQ140453992,IQ2118281,IQ24522,IQ333191,IQ875505,...,IQ20450986,IQ972190,IQ8548197,IQ9284076,IQ51508170,IQ22686070,IQ8033364,IQ1586158,IQ7664925,IQ61440355
1,IQ9454685,IQ252745708,IQ1856089,IQ28958586,IQ410392836,IQ140453992,IQ19053,IQ38026400,IQ104483226,IQ881385,...,IQ338321,IQ882318,IQ887231,IQ8225864,IQ873976,IQ226071413,IQ388030,IQ2118281,IQ4163040,IQ271680201
2,IQ691417,IQ4205366,IQ1856089,IQ62120637,0,IQ8548197,IQ60268423,IQ127436489,IQ333332,IQ536070194,...,IQ414192758,IQ140248389,IQ135285,IQ33604,IQ998257,IQ2118281,IQ173468401,IQ305013528,IQ6411769,IQ20991737
3,IQ41081420,0,IQ2483879,IQ140624293,0,IQ60927742,IQ998257,IQ388030,IQ252339,IQ78885253,...,IQ206492532,IQ4287992,IQ10004664,IQ518807,IQ20450986,IQ24913579,IQ96078,IQ19053,IQ701179,IQ1746003
4,IQ539299143,0,IQ24964899,IQ2118281,0,IQ224812364,IQ224101504,IQ24673899,IQ331893811,IQ2118281,...,IQ25304,IQ5489570,IQ2118281,IQ91868,IQ6520204,IQ141108755,IQ78881828,IQ873976,IQ144111,IQ7828330


In [21]:
sup_res = np.zeros((valid_count, valid_count))

for col in supply.columns:
    tic_name =  col.split(':')[1]
    if tic_name in ticker_list:
        #### get location
        x = ticker_list.index(tic_name)
        for c_id in supply[col]:
            try:
                y = ticker_list.index(map_dict[c_id])
                sup_res[x,y] = 1
            except:
                continue

In [22]:
        
fileObject = open(os.path.join(save_location, 'relations_author_source/supply_relation.pkl'), 'wb')

pickle.dump(sup_res,fileObject)
fileObject.close()

### Strategic Relationship

In [23]:
strategy = pd.read_csv(os.path.join(author_code, 'raw_data/relation/sp500_supply.tsv'), sep = '\t')

In [24]:
strategy.head()

,NYSE:MMM,NYSE:AOS,NYSE:ABT,NYSE:ABBV,NasdaqGS:ABMD,NYSE:ACN,NasdaqGS:ATVI,NasdaqGS:ADBE,NYSE:AAP,NasdaqGS:AMD,...,NasdaqGS:WYNN,NasdaqGS:XEL,NYSE:XRX,NasdaqGS:XLNX,NYSE:XYL,NYSE:YUM,NasdaqGS:ZBRA,NYSE:ZBH,NasdaqGS:ZION,NYSE:ZTS
0,IQ881351,IQ250510,IQ303225632,IQ3182703,IQ28046477,IQ140453992,IQ2118281,IQ24522,IQ333191,IQ875505,...,IQ20450986,IQ972190,IQ8548197,IQ9284076,IQ51508170,IQ22686070,IQ8033364,IQ1586158,IQ7664925,IQ61440355
1,IQ9454685,IQ252745708,IQ1856089,IQ28958586,IQ410392836,IQ140453992,IQ19053,IQ38026400,IQ104483226,IQ881385,...,IQ338321,IQ882318,IQ887231,IQ8225864,IQ873976,IQ226071413,IQ388030,IQ2118281,IQ4163040,IQ271680201
2,IQ691417,IQ4205366,IQ1856089,IQ62120637,0,IQ8548197,IQ60268423,IQ127436489,IQ333332,IQ536070194,...,IQ414192758,IQ140248389,IQ135285,IQ33604,IQ998257,IQ2118281,IQ173468401,IQ305013528,IQ6411769,IQ20991737
3,IQ41081420,0,IQ2483879,IQ140624293,0,IQ60927742,IQ998257,IQ388030,IQ252339,IQ78885253,...,IQ206492532,IQ4287992,IQ10004664,IQ518807,IQ20450986,IQ24913579,IQ96078,IQ19053,IQ701179,IQ1746003
4,IQ539299143,0,IQ24964899,IQ2118281,0,IQ224812364,IQ224101504,IQ24673899,IQ331893811,IQ2118281,...,IQ25304,IQ5489570,IQ2118281,IQ91868,IQ6520204,IQ141108755,IQ78881828,IQ873976,IQ144111,IQ7828330


In [25]:
strategy_res = np.zeros((valid_count, valid_count))

for col in strategy.columns:
    tic_name =  col.split(':')[1]
    if tic_name in ticker_list:
        #### get location
        x = ticker_list.index(tic_name)
        for c_id in strategy[col]:
            try:
                y = ticker_list.index(map_dict[c_id])
                strategy_res[x,y] = 1
            except:
                continue

In [26]:
strategy_res.sum(axis=1)

array([ 1.,  0.,  0.,  0.,  0.,  2.,  0.,  2.,  1.,  0.,  0.,  1.,  1.,
       13.,  0.,  0.,  2.,  0.,  0.,  1., 10.,  1., 13.,  2.,  0.,  3.,
        6.,  1.,  0.,  0.,  1.,  7.,  0.,  1.,  0.,  0.,  3.,  0.,  2.,
        0., 25.,  1.,  0.,  2.,  0.,  0., 10.,  1.,  2.,  1.,  0.,  1.,
        0.,  1., 13.,  0.,  0., 10.,  0.,  0.,  3.,  1.,  2.,  1.,  3.,
        0.,  0.,  1.,  4.,  7.,  1.,  2.,  1.,  4., 11.,  0.,  0.,  2.,
        6.,  0.,  5.,  0.,  0.,  0.,  1.,  0.,  0.,  6.,  0.,  0.,  3.,
        1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  2.,  8.,  1.,  0.,
        0.,  5.,  0.,  1.,  0.,  2.,  2.,  0.,  0.,  0.,  0.,  7.,  2.,
        3., 11.,  2.,  1.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  2.,  2.,  0.,  2.,  0.,  1., 16.,  0.,  1.,  2.,  1.,
        5.,  1.,  1.,  2.,  3.,  9.,  0., 11.,  7.,  2.,  2.,  0.,  3.,
        2.,  2.,  1.,  6.,  2.,  0.,  1.,  1.,  0.,  0.,  2.,  0.,  1.,
        4.,  0.,  4.,  0.,  0.,  0.,  0.,  2.,  3.,  0.,  3.,  0

In [27]:
fileObject = open(os.path.join(save_location, 'relations_author_source/strategic_relation.pkl'), 'wb')

pickle.dump(strategy_res,fileObject)
fileObject.close()

### competition

In [28]:
competition = pd.read_csv(os.path.join(author_code, 'raw_data/relation/sp500_cptr.tsv'), sep = '\t')

In [29]:
competition.head()

,NYSE:MMM,NYSE:AOS,NYSE:ABT,NYSE:ABBV,NasdaqGS:ABMD,NYSE:ACN,NasdaqGS:ATVI,NasdaqGS:ADBE,NYSE:AAP,NasdaqGS:AMD,...,NasdaqGS:WYNN,NasdaqGS:XEL,NYSE:XRX,NasdaqGS:XLNX,NYSE:XYL,NYSE:YUM,NasdaqGS:ZBRA,NYSE:ZBH,NasdaqGS:ZION,NYSE:ZTS
0,IQ32727633,IQ4427552,IQ289194,IQ30111163,IQ247483,IQ4532786,IQ5490183,IQ113275259,IQ24333,IQ24658,...,IQ648902,IQ289272,IQ4532786,IQ168864,IQ107678,IQ541884195,IQ289194,IQ247483,IQ47416993,IQ24160
1,IQ228503119,IQ528910168,IQ24160,IQ289194,IQ317627,IQ27166398,IQ348893,IQ308402,IQ8441642,IQ22816801,...,IQ370714,IQ312949,IQ60414516,IQ247347,IQ251148,IQ331010,IQ24567,IQ34670,IQ3115038,IQ247483
2,IQ4522840,IQ10437864,IQ24347,IQ247483,IQ697968,IQ362956,IQ310781228,IQ289194,IQ18749,IQ251411,...,IQ3133890,IQ856489,IQ144311584,IQ111231759,IQ717451,IQ24133,IQ24937,IQ967243,IQ27059842,IQ7665300
3,IQ289194,IQ4222516,IQ5875197,IQ141885706,IQ411220,IQ250079,IQ662734,IQ413289027,IQ4258476,IQ388030,...,IQ13103664,IQ135470,IQ972190,IQ32877378,IQ4527821,IQ160837,IQ171296,IQ30550,IQ19049,IQ1348220
4,IQ2878864,IQ959170,IQ25121580,IQ214816432,IQ134377,IQ829787,IQ134997,IQ7828060,IQ38769619,IQ302742004,...,IQ322193,IQ20725096,IQ1408776,IQ24658,IQ8033506,IQ4271690,IQ26399,IQ9405055,IQ2118281,IQ5920253


In [30]:
competition_res = np.zeros((valid_count, valid_count))

for col in competition.columns:
    tic_name =  col.split(':')[1]
    if tic_name in ticker_list:
        #### get location
        x = ticker_list.index(tic_name)
        for c_id in competition[col]:
            try:
                y = ticker_list.index(map_dict[c_id])
                competition_res[x,y] = 1
            except:
                continue

In [31]:
competition_res.sum(axis=1)

array([41.,  1., 45., 17.,  5., 28.,  5., 23.,  8.,  8., 13., 16.,  3.,
       15.,  5.,  7.,  8.,  1.,  2., 24., 42., 11., 52., 10.,  6., 23.,
       23., 18.,  2.,  3., 15.,  5., 24., 14., 10., 14.,  8., 11., 13.,
       11., 30.,  8., 11., 15.,  8., 13., 26.,  6., 10., 13.,  7., 12.,
       10.,  9., 26., 14., 33., 14., 28., 14., 11., 22., 16., 42., 14.,
        7., 11., 17., 18., 28.,  4.,  4., 19.,  7., 11., 10., 11., 15.,
       28.,  4., 11., 22.,  5., 10., 26., 10.,  7., 42., 23., 20., 18.,
        9., 11., 28., 18., 19., 13., 14., 10.,  3., 10., 20.,  2.,  4.,
       12., 12.,  2., 51.,  5.,  7., 12.,  7., 15., 12.,  7., 14., 23.,
       12., 11., 14., 12.,  5., 16.,  9., 14.,  2.,  8., 24., 12., 24.,
       16., 22.,  7.,  8., 28.,  9., 12., 23.,  4.,  9., 11.,  9., 10.,
        8.,  7.,  7., 20., 14., 27., 11., 10., 15., 39., 19., 13., 22.,
       33., 12.,  1.,  9., 12.,  5., 12., 12., 18.,  7., 10., 13., 25.,
       49., 18., 12., 17., 11., 28., 19.,  8.,  8.,  6.,  9., 10

In [32]:
fileObject = open(os.path.join(save_location, 'relations_author_source/competition_relation.pkl'), 'wb')

pickle.dump(competition_res,fileObject)
fileObject.close()

### customer

In [33]:
customer = pd.read_csv(os.path.join(author_code, 'raw_data/relation/sp500_cstm.tsv'), sep = '\t')

In [34]:
customer.head()

,NYSE:MMM,NYSE:AOS,NYSE:ABT,NYSE:ABBV,NasdaqGS:ABMD,NYSE:ACN,NasdaqGS:ATVI,NasdaqGS:ADBE,NYSE:AAP,NasdaqGS:AMD,...,NasdaqGS:WYNN,NasdaqGS:XEL,NYSE:XRX,NasdaqGS:XLNX,NYSE:XYL,NYSE:YUM,NasdaqGS:ZBRA,NYSE:ZBH,NasdaqGS:ZION,NYSE:ZTS
0,IQ881351,IQ180871,IQ655542,IQ24809,0,IQ289634104,IQ29096,IQ130202001,IQ313055,IQ23335330,...,IQ9916944,IQ140248389,IQ880771,IQ37256150,IQ3103613,IQ4481067,IQ395364,IQ9776743,IQ334518,IQ20475385
1,IQ881351,0,IQ7687519,IQ172207,0,IQ4502766,IQ18749,IQ25858,0,IQ28071600,...,IQ5410375,IQ22546470,IQ135470,IQ253018,IQ20411367,IQ137464369,IQ131464,IQ22001832,IQ363801,IQ162270
2,IQ7960453,0,IQ8623180,IQ206928583,0,IQ745257,IQ24937,IQ310862054,0,IQ29096,...,IQ882710,IQ32951,IQ23248034,IQ428613,0,IQ417092,IQ884042,0,IQ54842846,0
3,IQ656428779,0,IQ26051622,IQ10039157,0,IQ121238,IQ24937,IQ27878961,0,IQ24937,...,IQ23248134,IQ292770,IQ24771,IQ10332229,0,IQ540129360,IQ79069170,0,IQ701179,0
4,IQ41081420,0,IQ12168804,IQ288195,0,IQ691202,IQ255397,IQ332700,0,IQ554210686,...,IQ4266252,IQ12040739,IQ9637358,IQ266671679,0,IQ330280517,IQ173468401,0,IQ144111,0


In [35]:
customer_res = np.zeros((valid_count, valid_count))

for col in customer.columns:
    tic_name =  col.split(':')[1]
    if tic_name in ticker_list:
        #### get location
        x = ticker_list.index(tic_name)
        for c_id in customer[col]:
            try:
                y = ticker_list.index(map_dict[c_id])
                customer_res[x,y] = 1
            except:
                continue

In [36]:
customer_res.sum(axis=1)

array([ 1.,  1.,  0.,  4.,  0.,  1.,  8.,  1.,  1.,  3.,  0.,  1.,  0.,
       18.,  0.,  0.,  7.,  0.,  0.,  0.,  3.,  1.,  2.,  0.,  1.,  0.,
        4.,  0.,  3.,  0.,  0.,  1.,  0.,  5.,  1.,  1.,  0.,  1.,  0.,
        0.,  6.,  2.,  0.,  2.,  0.,  2.,  5.,  0.,  1.,  0.,  0.,  0.,
        0.,  3., 13.,  1.,  0.,  0.,  0.,  4.,  2.,  1.,  1.,  1.,  1.,
        0.,  1.,  2.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.,  1.,  0.,  3.,
        0.,  0.,  2.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  2.,  0.,
        2.,  1.,  0.,  0.,  0.,  0.,  2.,  0.,  1.,  0.,  0.,  8.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,  0.,  1.,  2.,  0.,  0.,
        0.,  0.,  3.,  4.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  2.,  0.,
        0.,  0.,  0.,  2.,  5.,  1.,  0., 13.,  2.,  0.,  2.,  0.,  0.,
        1.,  0.,  0.,  0.,  2.,  2.,  1.,  1.,  0.,  0.,  3.,  0.,  0.,
        2.,  2.,  0.,  0.,  3.,  3.,  0.,  5.,  7.,  0.,  0.,  0

In [37]:
fileObject = open(os.path.join(save_location, 'relations_author_source/customer_relation.pkl'), 'wb')

pickle.dump(customer_res,fileObject)
fileObject.close()

### All

In [38]:
all_relation = pickle.load(open(os.path.join(author_code, 'data/relations/all_relation.pkl'),'rb'))
competitor_relation = pickle.load(open(os.path.join(author_code, 'data/relations/competitor_relation.pkl'),'rb'))
customer_relation = pickle.load(open(os.path.join(author_code, 'data/relations/customer_relation.pkl'),'rb'))
industry_relation = pickle.load(open(os.path.join(author_code, 'data/relations/industry_relation.pkl'),'rb'))
strategic_relation = pickle.load(open(os.path.join(author_code, 'data/relations/stratigic_relation.pkl'),'rb'))
supply_relation = pickle.load(open(os.path.join(author_code, 'data/relations/supply_relation.pkl'),'rb'))

In [39]:
all_relation.shape

(198, 198)

In [40]:
temp = all_relation -\
np.where(competitor_relation+customer_relation+industry_relation+strategic_relation+supply_relation>0,1,0)

In [41]:
temp.sum(axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [42]:
all_res = np.where(ind_res + sup_res + strategy_res + competition_res + customer_res>0,1,0)

fileObject = open(os.path.join(save_location, 'relations_author_source/all_relation.pkl'), 'wb')

pickle.dump(all_res,fileObject)
fileObject.close()

In [43]:
pd.Series(ticker_list).to_csv(os.path.join(save_location, 'relations_author_source/company_list.csv'))